In [0]:
import os
import random
import pathlib
import tempfile
import time
import copy
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import classification_report
import numpy as np
from functools import partial
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [0]:
import torch
torch.cuda.empty_cache()
import gc

# Force garbage collection
gc.collect()


In [0]:
BASE_DIR= pathlib.Path('/Volumes/pmr_dev/lead_ingots/lead_ingot_images')
image_dir= BASE_DIR / 'Ingot'
label_dir= BASE_DIR / 'Labels/Labels.csv'
df = pd.read_csv(label_dir)
display(df)



In [0]:
file_names=df["Image"].tolist()
print(file_names)

In [0]:
# Load all images at once
def load_images(image_names):
    images = []
    for image_name in image_names:
        image_path = image_dir / image_name
        image = Image.open(image_path).convert('RGB')
        images.append(image)
    return images
# Load all images
images = load_images(file_names)

In [0]:
# Custom Dataset Class
class LeadIngotDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.labels = labels
        self.images = images  
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [0]:
transform = transforms.Compose([
    transforms.Resize((1024, 256)), #alexnet, vgg16 input size
    transforms.ToTensor()
])

In [0]:
new_df = df[["Image", "Goede blok"]]
new_df.columns = ['Image', 'label']

# Replace boolean or string values with integers
new_df['label'] = new_df['label'].replace({True: 1, False: 0})

print(new_df)


In [0]:
labels = new_df['label']
ds = LeadIngotDataset(images, labels, transform)

In [0]:
from torchvision import transforms
from PIL import Image
import random
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# Define the augmentation transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomVerticalFlip(),  # Randomly flip images vertically
    transforms.ToTensor()  # Convert images to tensor
])
to_pil = transforms.ToPILImage()
# Augment images labeled as '0' (Slecht)
def augment_data(dataset, labels, num_augmentations=5):
    augmented_images = []
    augmented_labels = []
    for i in range(len(dataset)):
        if labels[i] == 0:  # Only augment the images with label '0'
            image, label = dataset[i]
            
            # Generate augmentations
            for _ in range(num_augmentations):
                augmented_image = augmentation_transforms(to_pil(image))  # Apply the augmentation transforms
                augmented_images.append(augmented_image)
                augmented_labels.append(label)
    return augmented_images, augmented_labels

# Split dataset into train, validation, and test
train_val_indices, test_indices = train_test_split(
    range(len(ds)), test_size=0.2, stratify=labels, random_state=42
)

# Then split train+val into train and validation sets
train_indices, val_indices = train_test_split(
    train_val_indices, test_size=0.125, stratify=[labels[i] for i in train_val_indices], random_state=42
)

# Create Subset datasets for training, validation, and test
goede_train_dataset = Subset(ds, train_indices)
goede_val_dataset = Subset(ds, val_indices)
goede_test_dataset = Subset(ds, test_indices)

# Apply data augmentation on 'Slecht' class (label 0) for training set only
train_images, train_labels = [], []
for i in train_indices:
    train_images.append(ds[i][0])
    train_labels.append(labels[i])

augmented_images, augmented_labels = augment_data(goede_train_dataset, train_labels, num_augmentations=0)

# Combine augmented images with the original training images
combined_train_images = train_images + augmented_images
combined_train_labels = train_labels + augmented_labels

# Create a new augmented dataset for training
augmented_train_dataset = LeadIngotDataset(combined_train_images, combined_train_labels, transform=None)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(goede_val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(goede_test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Print statistics
train_labels = [labels[i] for i in train_indices]
val_labels = [labels[i] for i in val_indices]
test_labels = [labels[i] for i in test_indices]

print(f"Train set size: {len(augmented_train_dataset)} frequenties: {torch.bincount(torch.tensor(combined_train_labels))}")
print(f"Validation set size: {len(goede_val_dataset)} frequenties: {torch.bincount(torch.tensor(val_labels))}")
print(f"Test set size: {len(goede_test_dataset)} frequenties: {torch.bincount(torch.tensor(test_labels))}")


In [0]:
import torch.nn as nn
class simpleCNN (torch.nn.Module):
    def __init__(self):
        super(simpleCNN, self).__init__()
        self.encoder = nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(256 * 32 * 8, 128)
        self.fc2 = torch.nn.Linear(128, 2)
        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()

    def forward(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
    
        return x

In [0]:
model=simpleCNN()

device = torch.device("cuda:0")
# device = torch.device("cpu")

model = model.to(device)

In [0]:
x= torch.ones((32,3,1024,256))
x=x.to(device)
print(model(x).size())

In [0]:
print(torch.cuda.memory_allocated()/1000000000)  # Print the allocated memory
print(torch.cuda.memory_reserved()/1000000000)   # Print the reserved memory


In [0]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(model, num_epochs, train_dl, valid_dl, patience):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    patience_counter = 0
    best_model_wts = model.state_dict()  # Initialize best model weights

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        
        running_loss_train = 0.0
        correct_train = 0
        total_train = 0

        # Training loop
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()  # Zero gradients
            pred = model(x_batch)  # Forward pass
            loss = loss_fn(pred, y_batch)  # Calculate loss

            loss.backward()  # Backpropagation
            optimizer.step()  # Optimize weights

            running_loss_train += loss.item() * y_batch.size(0)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            correct_train += is_correct.sum().cpu()
            total_train += y_batch.size(0)

        loss_hist_train[epoch] = running_loss_train / total_train
        accuracy_hist_train[epoch] = correct_train / total_train

        # Validation loop
        model.eval()  # Set to evaluation mode
        running_loss_valid = 0.0
        correct_valid = 0
        total_valid = 0

        with torch.no_grad():  # Disable gradient computation
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)

                running_loss_valid += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                correct_valid += is_correct.sum().cpu()
                total_valid += y_batch.size(0)

        loss_hist_valid[epoch] = running_loss_valid / total_valid
        accuracy_hist_valid[epoch] = correct_valid / total_valid

        end_time = time.time()
        epoch_duration = end_time - start_time

        # Early stopping mechanism
        if epoch > 0 and loss_hist_valid[epoch] > min(loss_hist_valid[:epoch]):
            patience_counter += 1
        else:
            patience_counter = 0
            best_model_wts = model.state_dict()  # Save the best model weights

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            model.load_state_dict(best_model_wts)  # Load the best model weights
            break

        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {loss_hist_train[epoch]:.4f}, '
              f'Train Accuracy: {accuracy_hist_train[epoch]:.4f}, '
              f'Val Loss: {loss_hist_valid[epoch]:.4f}, '
              f'Val Accuracy: {accuracy_hist_valid[epoch]:.4f}, '
              f'Duration: {epoch_duration:.2f}s')

    return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid


num_epochs = 300
hist = train(model, num_epochs, train_loader, val_loader, patience=5)


In [0]:
# import time
# import torch
# import torch.nn as nn

# # Loss and optimizer setup
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# def overfit_single_batch(model, num_epochs, train_dl):
#     # Get a single batch from the training data
#     x_batch, y_batch = next(iter(train_dl))  # Take one batch
#     x_batch = x_batch.to(device)
#     y_batch = y_batch.to(device)
    
#     # Training loop (overfitting on the single batch)
#     for epoch in range(num_epochs):
#         start_time = time.time()
#         model.train()

#         optimizer.zero_grad()  # Zero gradients
#         pred = model(x_batch)  # Forward pass
#         loss = loss_fn(pred, y_batch)  # Compute loss

#         loss.backward()  # Backpropagate
#         optimizer.step()  # Update weights

#         # Print the loss for each epoch
#         print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")

#         end_time = time.time()
#         epoch_duration = end_time - start_time
#         print(f"Epoch Duration: {epoch_duration:.2f}s")

#     return model


# num_epochs = 100
# model = overfit_single_batch(model, num_epochs, train_loader)


In [0]:
torch.save(model.state_dict(), 'CNN.pth')

In [0]:
# Load the trained model
model = simpleCNN().to(device)
model.load_state_dict(torch.load('CNN.pth'))
model.eval()

In [0]:
from sklearn.metrics import classification_report

# Zet het model in evaluatiemodus
model.eval()

# Lijsten om de echte labels en voorspellingen op te slaan
all_labels = []
all_preds = []

# Geen gradientberekeningen nodig tijdens evaluatie
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Voorspellingen maken
        preds = model(x_batch)
        preds = torch.argmax(preds, dim=1)
        
        # Voeg de echte labels en voorspellingen toe aan de lijsten
        all_labels.extend(y_batch.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Genereer het classificatierapport
report = classification_report(all_labels, all_preds, target_names=['Slecht', 'Goed'])
print(report)

In [0]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Define the transformation for the images
transform = transforms.Compose([
    transforms.Resize((1024, 256)),  # Resizing the image
    transforms.ToTensor()            # Convert image to tensor
])

# Custom Dataset class to load images and make predictions
class ImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_path  # Return image and its path
import torch.nn as nn
class simpleCNN (torch.nn.Module):
    def __init__(self):
        super(simpleCNN, self).__init__()
        self.encoder = nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(256 * 32 * 8, 128)
        self.fc2 = torch.nn.Linear(128, 2)
        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()

    def forward(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
    
        return x
# Load the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = simpleCNN().to(device)
model.load_state_dict(torch.load('CNN.pth'))
model.eval()

# Path to the folder with images
image_folder = '/Volumes/pmr_dev/lead_ingots/lead_ingot_images/Ingot/Export20241129/'

# Get all image paths in the folder
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(('jpg', 'png', 'jpeg'))]

# Create dataset and dataloader for image evaluation
dataset = ImageDataset(image_paths, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# List to store images labeled as '0' (Slecht)
slecht_images = []

# Make predictions for each image
with torch.no_grad():
    for x_batch, image_paths_batch in dataloader:
        x_batch = x_batch.to(device)

        # Get model predictions
        preds = model(x_batch)
        preds = torch.argmax(preds, dim=1)

        # Add the images labeled as '0' (Slecht) to the list
        for i, pred in enumerate(preds):
            if pred.item() == 0:  # Label '0' corresponds to 'Slecht'
                slecht_images.append(image_paths_batch[i])
                print(image_paths_batch[i])

# Output the list of 'Slecht' images
print("Images labeled as 'Slecht' (0):")
for image_path in slecht_images:
    print(image_path)
